In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_3classes__Fourier.csv'
dataset_DS2 = pd.read_csv(os.path.join(rootDir, path, filename));

filename = 'MIT-BIH__DS1_3classes__Fourier.csv'
dataset_DS1 = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset_DS1.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34
0,0.140163,0.302116,0.256512,0.199795,0.167456,0.136428,0.092218,0.053257,0.035375,0.022090,...,0.001537,0.003202,0.000649,0.002842,0.002034,0.003303,0.002120,0.001724,0.001094,0
1,0.164694,0.294092,0.248227,0.189410,0.155726,0.119119,0.084469,0.051921,0.026894,0.022658,...,0.001776,0.001309,0.001864,0.002886,0.001600,0.001463,0.001042,0.000314,0.001563,0
2,0.264069,0.336712,0.279655,0.212017,0.169007,0.126995,0.081884,0.039922,0.025413,0.012168,...,0.002457,0.002108,0.001981,0.001673,0.002745,0.001638,0.001236,0.002879,0.002812,0
3,0.072350,0.351005,0.288275,0.218995,0.177219,0.124068,0.074372,0.044149,0.024169,0.016180,...,0.001813,0.001525,0.002232,0.003359,0.001211,0.002223,0.001470,0.002496,0.000781,0
4,0.029850,0.354420,0.295535,0.229964,0.184878,0.129478,0.076174,0.041396,0.027271,0.015194,...,0.002030,0.000666,0.001806,0.002329,0.001273,0.002032,0.001337,0.002298,0.001719,0


In [4]:
dataset_DS1.shape

(50969, 34)

# Data preprocessing

In [5]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

In this step we are going standardize our dataset. The pipeline.pkl will be loaded, and it contains the weights of normalization onf DS1 set.

In [6]:
data_std_DS1 = std_scaler.fit_transform(dataset_DS1.values[:,:-1])
data_label_DS1 = dataset_DS1.values[:,-1]

data_std_DS2 = std_scaler.transform(dataset_DS2.values[:,:-1])
data_label_DS2 = dataset_DS2.values[:,-1]

# Load classifiers

In [7]:
from sklearn.externals import joblib

random_search_SVM = joblib.load('./Models/classifiers_SVM.pkl')
random_search_MLP = joblib.load('./Models/classifiers_MLP.pkl')
random_search_Bayes = joblib.load('./Models/classifiers_MLP_Bayes.pkl')

svm_clf = random_search_SVM['SVM-RBF'].best_estimator_
mlp_clf = random_search_MLP['MLP'].best_estimator_
bayes_clf = random_search_Bayes['Naive-Bayes']

classifiers = {'MLP-768': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# TRAIN on DS1 and **TEST** on DS2 set

In [8]:
clf_outputs = {'true': dict((k, {}) for k in classifiers.keys()), 
               'pred': dict((k, {}) for k in classifiers.keys())}
results = dict((k, {}) for k in classifiers.keys())

for clf in classifiers.keys():
    classifiers[clf].fit(data_std_DS1, data_label_DS1)
        
    clf_outputs['true'][clf][0] = data_label_DS2
    clf_outputs['pred'][clf][0] = classifiers[clf].predict(data_std_DS2)
    
    print(clf) 

MLP-768
SVM-RBF
Naive-Bayes


IndexError: index 3 is out of bounds for axis 1 with size 3

# Save results to CSVs and figures

In [13]:
clf_outputs

{'true': {'MLP-768': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'SVM-RBF': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'Naive-Bayes': {0: array([0., 0., 0., ..., 0., 0., 0.])}},
 'pred': {'MLP-768': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'SVM-RBF': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'Naive-Bayes': {0: array([1., 1., 1., ..., 0., 0., 1.])}}}

In [15]:
results = results_clf(3, clf_outputs['true'], clf_outputs['pred'])

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_12_scikitLearn__allCLF_onDS2_allFeatures_3Classes/experiment.py:150: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)


In [16]:
export_results(results, 'validation')

MLP-768
SVM-RBF
Naive-Bayes


/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_12_scikitLearn__allCLF_onDS2_allFeatures_3Classes/experiment.py:286: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)


In [18]:
save_models(classifiers)
save_pipeline(std_scaler)